In [1]:
#from nltk import download

#download()

In [2]:
from nltk import sent_tokenize
from gensim.utils import tokenize

with open('quaran.txt') as f:
    text = f.read().replace('\n', '')
    
sentences = [list(tokenize(sent)) for sent in sent_tokenize(text)]

In [3]:
from gensim.models import *

min_count = 1
size = 50
window = 10
 
model = Word2Vec(sentences, min_count=min_count, size=size, window=window,  workers=4)

In [4]:
try:
    print(model.most_similar('Allah'))
except KeyError:
    print('No such key!')

[('atom', 0.9004787802696228), ('grace', 0.8910990357398987), ('Him', 0.888778030872345), ('servants', 0.8792895674705505), ('mercy', 0.8764598369598389), ('Eat', 0.8549256920814514), ('best', 0.8515028953552246), ('wrongdoers', 0.8507567048072815), ('Friend', 0.8491989374160767), ('path', 0.8486315608024597)]
